# Modelado "forward"

`
Autores:
Brigitte Aguilar, Sofía Poux, Elizabeth Young
`

Modificado de *PracticalMEEG2022: MNE-python hands-on tutorial*. Por Britta Westner.

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt

import os
import mne

# Cambia la siguiente dirección en función de donde se encuentra la carpeta `Datos`
data_path = os.path.expanduser("C:\MNE_projects\mne_CuttingGardens_OroVerde\Datos")

# Cambia la siguiente dirección en función de donde se encuentra la carpeta `extra_data_mne`
extra_path = os.path.expanduser("C:\MNE_projects\mne_CuttingGardens_OroVerde\Datos\extra_data_mne")

raw_fname = os.path.join(data_path,
   'sub-01\sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

# Calcular el operador 'forward'

Para calcular un operador 'forward' (también conocido como modelo 'forward', matriz de ganancia o, con menos precisión, campo principal) necesitamos:
   - un archivo ``-trans.fif`` que contenga la información de corregistro
   - un espacio fuente
   - las superficies BEM

## Calcular y visualizar superficies BEM


Aquí trabajamos con **superficies BEM precalculadas**.

En caso de que quieras calcular las superficies BEM, podrías realizarlo mediante FreeSurfer (https://surfer.nmr.mgh.harvard.edu/fswiki/DownloadAndInstall#Download) y utiliza cualquiera de las dos siguientes herramientas de línea de comandos:

[mne watershed_bem](http://martinos.org/mne/dev/generated/commands.html#mne-watershed-bem)

[mne flash_bem](http://martinos.org/mne/dev/generated/commands.html#mne-flash-bem)

O se puede hacer directamente llamando a las funciones (esta opción requiere la instalación de FreeSurfer):

https://mne.tools/stable/generated/mne.bem.make_watershed_bem.html

https://mne.tools/stable/generated/mne.bem.make_flash_bem.html


Si deseas probar la reconstrucción BEM por tu cuenta más adelante, podes instalar FreeSurfer en MacOS o Linux directamente y en una VirtualMachine para Windows.

Para calcular modelos BEM para este conjunto de datos, se configuró `SUBJECTS_DIR` y se ejecutó:

    mne watershed_bem -s sub-01 --overwrite
    mne make_scalp_surfaces -s sub-01 --force --overwrite
    
Entonces, veamos primero las superficies BEM.

Para EEG se utilizaron 3 capas (cráneo interno, cráneo externo y piel), mientras que para MEG 1 capa (cráneo interno) es suficiente.

In [ ]:
# primero establecemos la ruta a la T1
t1_fname = os.path.join(extra_path, 'freesurfer/sub-01/mri/T1.mgz')

Podemos echar un vistazo a la resonancia magnética usando `nilearn`:

In [ ]:
%matplotlib inline
from nilearn import plotting
plotting.plot_anat(t1_fname);
plt.show()

También podemos mirar el modelo BEM (las superficies identificadas del BEM):

In [ ]:
# Esta es la ruta que apunta al directorio que tiene las carpetas temáticas para los modelos BEM.
# Aquí, es la ruta en la que se guardaron los datos adicionales.
subjects_dir = os.path.join(extra_path, 'freesurfer')

Dado que el dataset no contiene el modelo del sujeto, se utiliza un modelo 'promedio' que brinda MNE

In [ ]:
%matplotlib qt
# mne.viz.plot_bem(subject='sub-01', subjects_dir=subjects_dir,
#                  mri=t1_fname,
#                  orientation='coronal');

mne.viz.plot_bem(subject='fsaverage', subjects_dir=subjects_dir,
                 brain_surfaces="white",
                 orientation='coronal');

## Corregistro

El siguiente paso normalmente sería corregistrar el sistema de coordenadas de MRI con el sistema de coordenadas MEG. Esto se hace para que los sensores tengan la relación correcta con el modelo de cabeza para el cálculo del modelo directo.

Para este conjunto de datos, esto ya está hecho, pero si quisieras hacerlo, este es el código a usar:

In [ ]:
# mne.gui.coregistration(subject='sub-01', subjects_dir=subjects_dir, inst=raw_fname);

### Visualizando el corregistro

El corregistro es la operación que permite posicionar el cabezal y los sensores en un sistema de coordenadas común.

En MNE, la transformación para alinear el cabezal y los sensores se almacena en un archivo llamado *trans*. Es un archivo FIF que termina en `-trans.fif`. Se puede obtener con ``mne_analyze`` (herramientas Unix), ``mne.gui.coregistration`` (en Python, ver arriba) o mrilab si estás usando un sistema Neuromag.

Para la versión de Python, consulte la documentación en https://mne.tools/dev/generated/mne.gui.coregistration.html. También hay un enlace de vídeo que muestra cómo realizar el corregistro. Como asumimos que el corregistro está realizado, simplemente verificamos visualmente la alineación con el siguiente código.

In [ ]:
#subject = 'sub-01' --> se utilizaría si se contara con los modelos del sujeto, en este caso usamos el sujeto 'promedio'
subject = 'fsaverage'

trans_fname = os.path.join(extra_path, 'freesurfer/fsaverage/bem/fsaverage-trans.fif')
info = mne.io.read_info(raw_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=True,
                             subjects_dir=subjects_dir, verbose=True);

## Calcular el espacio fuente

El espacio fuente define las posiciones de las ubicaciones de fuentes candidatas. El siguiente código calcula dicho espacio fuente con una resolución OCT-6.

Tenga en cuenta que este es un espacio fuente de superficie (no de volumen).

In [ ]:
mne.set_log_level('WARNING')

src = mne.setup_source_space(subject, spacing='oct6',
                             subjects_dir=subjects_dir,
                             add_dist=False)

In [ ]:
src

`src` contiene dos partes, una para el hemisferio izquierdo (4098 ubicaciones) y otra para el hemisferio derecho (4098 ubicaciones).

Podemos visualizar el espacio fuente junto con el modelo de cabeza y también junto con el casco MEG.

In [ ]:
%matplotlib inline
mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=False, src=src,
                             subjects_dir=subjects_dir, verbose=True, meg=False,
                             eeg=False);

In [ ]:
mne.viz.plot_alignment(info, trans_fname, subject=subject,
                       src=src, subjects_dir=subjects_dir, dig=True,
                       surfaces=['head-dense', 'white'], coord_frame='meg')

### Calcular la solución directa

Ahora tenemos todos los ingredientes para calcular la solución 'forward', solo falta calcular el modelo BEM.

En este caso, utilizaremos el modelo BEM que brinda MNE para un sujeto 'promedio'.

De todas formas, se puede calcular un BEM para un sujeto específico de las tres capas (cráneo interno, cráneo externo y piel) o de una sola para disminuir el costo computacional (solo cráneo interno) que luego se podrá usar para MEG (pero no EEG).

Primero, se calcula el modelo BEM usando `mne.make_bem_solution()`, luego la solución 'forward' usando `mne.make_forward_solution()`.

Los comandos para el cálculo de los modelos para un sujeto específico se dejan comentados a continuación.

In [ ]:
# conductivity = (0.3,)  # for single layer
# # conductivity = (0.3, 0.006, 0.3)  # for three layers
# model = mne.make_bem_model(subject=subject, ico=4,
#                            conductivity=conductivity,
#                            subjects_dir=subjects_dir)
# bem = mne.make_bem_solution(model)
bem_path = os.path.join(extra_path, 'freesurfer/fsaverage/bem/fsaverage-5120-5120-5120-bem-sol.fif')

bem = mne.read_bem_solution(bem_path)


En caso de generar un modelo BEM, se podría guardar la solución en el disco, para utilizarlo mas adelante.
En nuestro caso, utilizamos un archivo ya cargado por lo que no necesitamos dichos comandos, por lo que se dejan comentados.

In [ ]:
# bem_fname = os.path.join(extra_path,
#     'source_recon/sub-01/sub-01-bem.fif')
# mne.bem.write_bem_solution(bem_fname, bem, overwrite=True)

Ahora calculamos el modelo 'forward'

Paciencia..el cómputo podría tomar su tiempo!

In [ ]:
fwd = mne.make_forward_solution(raw_fname, trans=trans_fname,
                                src=src, bem=bem,
                                meg=True,  # incluimos los canales MEG
                                eeg=True,  # excluimos los canales EEG
                                mindist=5.0,  # ignoramos superficies <= 5mm del cráneo interno
                                n_jobs=1)  # numero de procesos que corren en paralelo

In [ ]:
fwd

In [ ]:
leadfield = fwd['sol']['data']
print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)

También guardamos el modelo 'forward' para su uso posterior:

In [ ]:
fwd_fname = os.path.join(extra_path, 'source_recon/sub-01/sub-01-meg-fwd.fif')
mne.write_forward_solution(fwd_fname, fwd, overwrite=True)

## Calcular y mostrar mapas de sensibilidad

Calculemos un mapa de sensibilidad para gradiómetros y fijamos la orientación del dipolo.

In [ ]:
sens_map = mne.sensitivity_map(fwd, ch_type='grad', mode='fixed')

Para visualizar en 3D:

In [ ]:
# habilitar el backend correcto para el trazado 3D

clim = dict(kind='percent', lims=(0.0, 50, 99), smoothing_steps=3)  # veamos dipolos individuales
brain = sens_map.plot(subject=subject, time_label='GRAD sensitivity',
                      subjects_dir=subjects_dir, clim=clim, smoothing_steps=8);
view = 'lat'
brain.show_view(view)


<div class="alert alert-success">
    <b>EJERCICIO</b>:
      <ul>
        <li>Calcula y traza los mapas de sensibilidad para magnetómetros y compárelo con los gradiómetros. </li>
        <li>¿Podés justificar las afirmaciones de que MEG no es sensible a fuentes radiales?</li>
        <li>Intentá cambiar el parámetro `mode` en `mne.sensitivity_map` y observa cómo el operador de orientación hacia adelante cambia las imágenes según se deja libre o fijo.</li>
      </ul>
</div>